In [1]:
# =============================
# LIGHTGBM: обучение и инференс
# =============================
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import joblib

# ==== 1. Загрузка данных ====
# Предположим, что у нас есть CSV с фичами и таргетом
DATA_PATH = "data.csv"  # <-- путь к вашему файлу

df = pd.read_csv(DATA_PATH)
print("Данные загружены:", df.shape)
print(df.head())

# ==== 2. Разделение на признаки и цель ====
target_col = "target"  # <-- замените на имя вашей целевой колонки
X = df.drop(columns=[target_col])
y = df[target_col]

# ==== 3. Разделение на train / valid ====
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ==== 4. Создание датасетов LightGBM ====
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

# ==== 5. Настройки модели ====
params = {
    "objective": "regression",       # или 'binary', 'multiclass'
    "metric": "rmse",                # метрика (rmse, binary_logloss, multi_logloss)
    "boosting_type": "gbdt",         # 'gbdt', 'dart', 'goss'
    "learning_rate": 0.05,
    "num_leaves": 31,
    "max_depth": -1,
    "verbose": -1,
}

# ==== 6. Обучение ====
print("Обучение модели...")
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, valid_data],
    valid_names=["train", "valid"],
    num_boost_round=500,
    early_stopping_rounds=50,
    verbose_eval=50
)

# ==== 7. Сохранение модели ====
joblib.dump(model, "lightgbm_model.pkl")
print("Модель сохранена в 'lightgbm_model.pkl'")

# ==== 8. Инференс (предсказание на новых данных) ====
print("Инференс на валидации...")
y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

# ==== 9. Метрика ====
if params["objective"] == "regression":
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    print(f"RMSE: {rmse:.4f}")
else:
    y_pred_labels = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_valid, y_pred_labels)
    print(f"Accuracy: {acc:.4f}")

# ==== 10. Пример предсказания на новых данных ====
new_data = X_valid.iloc[:5]
new_pred = model.predict(new_data)
print("Пример предсказаний:\n", new_pred)


FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [5]:
# =============================
# Генерация данных + LIGHTGBM
# =============================
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

# ==== 1. Генерация синтетических данных ====
# Пусть у нас задача регрессии с 20 фичами
np.random.seed(42)
n_samples = 5000
n_features = 20

# Фичи
X = np.random.randn(n_samples, n_features)

# Генерируем цель как линейную комбинацию + шум
true_w = np.random.randn(n_features)
y = X @ true_w + np.random.randn(n_samples) * 0.5

# Создаём DataFrame, как будто мы загрузили его из CSV
df = pd.DataFrame(X, columns=[f"f{i}" for i in range(n_features)])
df["target"] = y

print("Синтетические данные созданы:", df.shape)
print(df.head())

# ==== 2. Разделение на X и y ====
X = df.drop(columns=["target"])
y = df["target"]

# ==== 3. Train/valid split ====
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ==== 4. LightGBM dataset ====
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

# ==== 5. Параметры модели ====
params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "verbose": -1,
}

# ==== 6. Обучение ====
print("\nОбучение модели LightGBM...")
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, valid_data],
    valid_names=["train", "valid"],
    num_boost_round=300
)

# ==== 7. Сохранение модели ====
joblib.dump(model, "lightgbm_model.pkl")
print("\nМодель сохранена: lightgbm_model.pkl")

# ==== 8. Инференс ====
print("\nИнференс...")
y_pred = model.predict(X_valid)

rmse = mean_squared_error(y_valid, y_pred)
print(f"RMSE: {rmse:.4f}")

# ==== 9. Тестовое предсказание на новых данных ====
new_sample = X_valid.iloc[:3]
new_pred = model.predict(new_sample)

print("\nПример предсказаний для новых данных:")
print(new_pred)


Синтетические данные созданы: (5000, 21)
         f0        f1        f2        f3        f4        f5        f6  \
0  0.496714 -0.138264  0.647689  1.523030 -0.234153 -0.234137  1.579213   
1  1.465649 -0.225776  0.067528 -1.424748 -0.544383  0.110923 -1.150994   
2  0.738467  0.171368 -0.115648 -0.301104 -1.478522 -0.719844 -0.460639   
3 -0.479174 -0.185659 -1.106335 -1.196207  0.812526  1.356240 -0.072010   
4 -0.219672  0.357113  1.477894 -0.518270 -0.808494 -0.501757  0.915402   

         f7        f8        f9  ...       f11       f12       f13       f14  \
0  0.767435 -0.469474  0.542560  ... -0.465730  0.241962 -1.913280 -1.724918   
1  0.375698 -0.600639 -0.291694  ...  1.852278 -0.013497 -1.057711  0.822545   
2  1.057122  0.343618 -1.763040  ... -0.385082 -0.676922  0.611676  1.031000   
3  1.003533  0.361636 -0.645120  ...  1.538037 -0.035826  1.564644 -2.619745   
4  0.328751 -0.529760  0.513267  ...  0.968645 -0.702053 -0.327662 -0.392108   

        f15       f16      